# Tutorial: Poisson equation with multiple boundary conditions

In this tutorial, we will learn

   -  How to solve a simple PDE in Julia with Gridap
   -  How to load a discrete model (aka a FE mesh) from a file
   -  How to build a conforming Lagrangian FE space
   -  How to define the different terms in a weak form
   -  How to define inhomogeneous Neumann and Robin boundary conditions

We start from the following boundary problem:
find the scalar field $u$ such that

$$
\left\lbrace
\begin{aligned}
-\Delta u = f  \ &\text{in} \ \Omega,\\
u = g_D \ &\text{on}\ \Gamma_{\rm D},\\
\nabla u\cdot n = g_N \ &\text{on}\  \Gamma_{\rm N}.\\
\end{aligned}
\right.
$$
Here, $n$ denotes the outwards unit normal vector to the Neumann
boundary $\Gamma_{\rm N} = \{0\} \times [0,1] \cup \{1\} \times [0,1]$ and $\Gamma_D = \partial\Omega \setminus\Gamma_{\rm N}$. 

## Derivation of the weak formulation
 First we need to derive the weak form associated with mixed boundary problem above.
 This is basically a combination of how you have derived weak formulations for the Poisson problem with pure Dirichlet or pure Neumann boundary condition.

For the Dirichlet (essential) boundary conditions, we once again build the boundary conditions into the trial function space
$V_g$ and its corresponding test function space $V_0$: 

\begin{align*}
V_g &:= H^1_{g, \Gamma_{\rm D}}(\Omega) :=  \{v \in H^1(\Omega) : v|_{\Gamma_{\rm D}} = g \}, \\
V_0 &:= H^1_{0, \Gamma_{\rm D}}(\Omega) :=  \{v \in H^1(\Omega) : v|_{\Gamma_{\rm D}} = 0 \}.
\end{align*}

Now multiplying $-\Delta u = f$ with a test function $v \in V_0$ we obtain

\begin{align}
\int_{\Omega} fv \;{\rm d}x 
&= 
-\int_{\Omega} \Delta u v \;{\rm d}x 
\\
&=
\int_{\Omega}  \nabla u \cdot \nabla v \;{\rm d}x
-\int_{\Gamma} \partial_n u v \;{\rm d}S 
\\
&=
\int_{\Omega}  \nabla u \cdot \nabla v \;{\rm d}x
-\int_{\Gamma_{\rm D}}  \partial_n u v \;{\rm d}S
-\int_{\Gamma_{\rm N}}  \partial_n u v \;{\rm d}S
\end{align}
Now the first surface integral 
$\int_{\Gamma_{\rm D}}  \partial_n u v \;{\rm d}S$
vanishes since the test function $v\in V_0$ is $0$ on the Dirichlet boundary.
For the second surface integral, we can insert the natural boundary condition
$\nabla u\cdot n = \partial_n u = g_N$ to obtain the linear form   
$-\int_{\Gamma_{\rm N}}  g_N v \;{\rm d}S$ which only depends on the test function $v$.
Incorporating this contribution into the linear form thus leads us to the formulation:

Find $u\in V_g$ such that 
 $$ a(u,v) = l(v) $$
where the bilinear and linear forms are given by 
\begin{align*}
  a(u,v) &\doteq \int_{\Omega} \nabla v \cdot \nabla u \ {\rm d}\Omega, \\
   l(v) &\doteq \int_{\Omega} v\ f  \ {\rm  d}\Omega + \int_{\Gamma_{\rm N}} v\ g_N \ {\rm d}\Gamma_{\rm N}.
\end{align*}


### Numerical scheme
 To solve this PDE, we use a conventional Galerkin finite element (FE) method with conforming Lagrangian FE spaces (see, e.g., [1] for specific details on this formulation). 
We will again use the *method of manufactured solution* and construct two test problems, both posed on $\Omega = [-1,1]^2$. 
For the two test problems (TP), we choose the exact solution $u_0$ and $u_2$ to be 
\begin{align*}
(TP0)\quad u_1(x,y) &= x + y,
\\
(TP1)\quad u_2(x,y) &= \sin(2 \pi x) \cos( 2 \pi y) + u_1(x, y).
\end{align*}

The problem will be solved numerically by approximating the spaces $U_g$ and $V_0$ by their discrete counterparts associated with a FE mesh of the computational domain $\Omega$. As we have anticipated, we consider standard conforming Lagrangian FE spaces for this purpose.

### Task 1

Compute $f$, $g_D$ and $g_N$ for both test problems.

**Solution.**

#### TP1

\begin{align*}
f &= 
\\
g_N &=
\end{align*}

#### TP2

\begin{align*}
f &= 
\\
g_N &=
\end{align*}

## Using boundary tags
Next, we learn how to use and manipulate boundary tags. Typically when a mesh is generated, parts of the domain or boundary are tagged using some labels
in order to be able to distinguish them in the implementation of the numerical scheme.
In our case we have to distinguish between the Dirichlet boundary and the Neumann boundary as
the Dirichlet boundary condition need to be incorporated into the discrete function spaces while the Neumann boundary needs to be accounted for
in the definition of the linear form $l(\cdot)$.

Let's make again a triangular mesh for the unit square and plot it.

In [ ]:
using Gridap

domain = (0, 1, 0, 1)
partition = nx, ny = 4, 4
model = CartesianDiscreteModel(domain, partition)


Remember that for the plotting of $\Omega$, we need to create a `Triangulation` object from 
which we can plot using `GridapMakie`.

In [ ]:
using GridapMakie, CairoMakie, FileIO
Ω = Triangulation(model)
fig = plot(Ω)
wireframe!(Ω, color=:black)
scatter!(Ω, marker=:circle, markersize=20, color=:blue)
display(fig)
save("mesh_nx_$(nx)_ny_$(ny).svg", fig)

### Task 2
Let's visualize the boundary in the same manner, but use a `BoundaryTriangulation` instead of a `Triangulation` type.

**Solution.**

In [ ]:
Γ = BoundaryTriangulation(model)
fig = plot(Γ)
wireframe!(Γ, color=:black)
scatter!(Γ, marker=:circle, markersize=20, color=:blue)
display(fig)

The discrete `model` we just generated also contains some extra labels which keep track of e.g. various sides of the model.
The `tags` or `labels` can be extracted using the `get_face_labeling` function like this:
 ```julia
labels = get_face_labeling(model)
 ```
 Afterwards, you either inspects in e.g. the workspace viewer in Visual Studio Code or 
 by simply accessing and printing its `tag_to_name` member. 
 Those tags related entities can also be easily visualized if you write the model
 to a vtk file and load it into Paraview.

In [ ]:
labels = get_face_labeling(model)
labels.tag_to_name

The tags can be used to extract various parts of the boundary
using 
```julia
Γ_tag = BoundaryTriangulation(model, tags=["TAGNAME"])

```

### Task 3
Use that and visualize/plot each parts of the "boundary" which are associated with each tag.
Note that the "tag_1" to "tag_4" are actually tags associated with vertices and not with faces.


**Solution.**


### Task 4

Before we turn to the numerical solution of the full Poisson problem, we do a 
little preliminary exercise.

Integrate the function $g(x,y) = x + 2y$ 
separately over the left, right, bottom and top boundary of the boundary (check the results!).


### Task 5

Now, based on the previous tutorial and this tutorial, 
implemented a FEM solver. Use (TP1) and (TP2) to 
check your implementation, including a convergence test.

## Homework

Now following the steps in this tutorial, implement a FEM-based solver for the Poisson problem 
$$
\left\lbrace
\begin{aligned}
-\kappa \Delta u = f  \ &\text{in} \ \Omega,\\
u = g_D \ &\text{on}\ \Gamma_{\rm D},\\
\kappa \nabla u\cdot n = g_N \ &\text{on}\  \Gamma_{\rm N},\\
\kappa \nabla u\cdot n = \dfrac{1}{\epsilon}(g_R - u) \ &\text{on}\  \Gamma_{\rm R},\\
\end{aligned}
\right.
$$
where
$\Gamma_{\rm N} = \{0\} \times [0,1]$, 
$\Gamma_{\rm R} = \{1\} \times [0,1]$, 
and $\Gamma_D = \partial\Omega \setminus(\Gamma_{\rm N} \cup \Gamma_{\rm R})$. 

In particular this amounts to 
* determine the correct bilinear and linear forms
* use the method of manufactured solution to verify the correctness of your code. 
  Use at least one new, non-trivial manufactured solution.

We ask you to present your code/results in the tutorial session on Friday, 4th of March.

#### Happy coding! 